In [ ]:
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox
import pickle
import pandas as pd
import numpy as np

# Load the model
model = pickle.load(open("model.pkl", "rb"))

# Function to perform feature engineering and prediction
def predict_price():
    try:
        # Extracting data from entry fields
        journey_date = entry_journey_date.get()
        dep_time = entry_dep_time.get()
        arrival_time = entry_arrival_time.get()
        total_stops = int(entry_total_stops.get())
        airline = combo_airline.get()
        source = combo_source.get()
        destination = combo_destination.get()

        # Feature Engineering
        journey_day = int(pd.to_datetime(journey_date).day)
        journey_month = int(pd.to_datetime(journey_date).month)
        dep_hour = int(pd.to_datetime(dep_time).hour)
        dep_minute = int(pd.to_datetime(dep_time).minute)
        arrival_hour = int(pd.to_datetime(arrival_time).hour)
        arrival_minute = int(pd.to_datetime(arrival_time).minute)
        total_duration_minutes = (arrival_hour * 60 + arrival_minute) - (dep_hour * 60 + dep_minute)
        if total_duration_minutes < 0:
            total_duration_minutes += 24 * 60

        # One-hot encoding
        airline_dict = {'Jet Airways': 0, 'IndiGo': 1, 'Air India': 2, 'Multiple carriers': 3,
                        'SpiceJet': 4, 'Vistara': 5, 'GoAir': 6, 'others': 7}
        source_dict = {'Delhi': 0, 'Kolkata': 1, 'Mumbai': 2, 'Chennai': 3,
                       'Bangalore': 4, 'Hyderabad': 5, 'Cochin': 6}
        destination_dict = {'Cochin': 0, 'Delhi': 1, 'Hyderabad': 2, 'Kolkata': 3,
                            'Bangalore': 4, 'Mumbai': 5, 'Chennai': 6}

        input_data = {
            'Total_Stops': total_stops,
            'Journey_Day': journey_day,
            'Journey_Month': journey_month,
            'Dep_Hour': dep_hour,
            'Dep_Minute': dep_minute,
            'Arrival_Hour': arrival_hour,
            'Arrival_Minute': arrival_minute,
            'Total_Duration_minutes': total_duration_minutes,
            'Airline': airline_dict[airline],
            'Source': source_dict[source],
            'Destination': destination_dict[destination]
        }

        # Add missing columns with 0
        columns = [
            'Total_Stops', 'Journey_Day', 'Journey_Month', 'Dep_Hour', 'Dep_Minute',
       'Arrival_Hour', 'Arrival_Minute', 'Total_Duration_minutes',
       'Airline_Air India', 'Airline_GoAir', 'Airline_IndiGo',
       'Airline_Jet Airways', 'Airline_Multiple carriers', 'Airline_SpiceJet',
       'Airline_Vistara', 'Airline_others', 'Source_Chennai', 'Source_Delhi',
       'Source_Kolkata', 'Source_Mumbai', 'Destination_Cochin',
       'Destination_Delhi', 'Destination_Hyderabad', 'Destination_Kolkata',
       'Outlier_Yes', 'Journey_Day_of_Week_Monday',
       'Journey_Day_of_Week_Saturday', 'Journey_Day_of_Week_Sunday',
       'Journey_Day_of_Week_Thursday', 'Journey_Day_of_Week_Tuesday',
       'Journey_Day_of_Week_Wednesday'
        ]

        input_array = np.zeros(len(columns))
        for i, col in enumerate(columns):
            if col.startswith('Airline_'):
                airline_col = col.split('_')[1]
                if airline_col == airline:
                    input_array[i] = 1
            elif col.startswith('Source_'):
                source_col = col.split('_')[1]
                if source_col == source:
                    input_array[i] = 1
            elif col.startswith('Destination_'):
                destination_col = col.split('_')[1]
                if destination_col == destination:
                    input_array[i] = 1
            else:
                if col in input_data:
                    input_array[i] = input_data[col]

        # Predict the price
        prediction = model.predict([input_array])
        output = round(prediction[0], 2)

        # Display the result
        result_var.set(f"Your Flight price is Rs. {output}")

    except Exception as e:
        messagebox.showerror("Error", f"Error occurred: {e}")

# Create Tkinter window
root = tk.Tk()
root.title("Flight Price Prediction")
root.geometry("400x400")

# Create entry fields and labels
tk.Label(root, text="Journey Date (YYYY-MM-DD)").grid(row=0, column=0, padx=10, pady=5)
entry_journey_date = tk.Entry(root)
entry_journey_date.grid(row=0, column=1, padx=10, pady=5)

tk.Label(root, text="Departure Time (HH:MM)").grid(row=1, column=0, padx=10, pady=5)
entry_dep_time = tk.Entry(root)
entry_dep_time.grid(row=1, column=1, padx=10, pady=5)

tk.Label(root, text="Arrival Time (HH:MM)").grid(row=2, column=0, padx=10, pady=5)
entry_arrival_time = tk.Entry(root)
entry_arrival_time.grid(row=2, column=1, padx=10, pady=5)

tk.Label(root, text="Total Stops").grid(row=3, column=0, padx=10, pady=5)
entry_total_stops = tk.Entry(root)
entry_total_stops.grid(row=3, column=1, padx=10, pady=5)

tk.Label(root, text="Airline").grid(row=4, column=0, padx=10, pady=5)
combo_airline = ttk.Combobox(root, values=[
    'Jet Airways', 'IndiGo', 'Air India', 'Multiple carriers', 
    'SpiceJet', 'Vistara', 'GoAir', 'others'])
combo_airline.grid(row=4, column=1, padx=10, pady=5)
combo_airline.current(0)

tk.Label(root, text="Source").grid(row=5, column=0, padx=10, pady=5)
combo_source = ttk.Combobox(root, values=[
    'Delhi', 'Kolkata', 'Mumbai', 'Chennai', 'Bangalore', 'Hyderabad', 'Cochin'])
combo_source.grid(row=5, column=1, padx=10, pady=5)
combo_source.current(0)

tk.Label(root, text="Destination").grid(row=6, column=0, padx=10, pady=5)
combo_destination = ttk.Combobox(root, values=[
    'Cochin', 'Delhi', 'Hyderabad', 'Kolkata', 'Bangalore', 'Mumbai', 'Chennai'])
combo_destination.grid(row=6, column=1, padx=10, pady=5)
combo_destination.current(0)

# Create predict button and result label
btn_predict = tk.Button(root, text="Predict Price", command=predict_price)
btn_predict.grid(row=7, column=0, columnspan=2, pady=20)

result_var = tk.StringVar()
tk.Label(root, textvariable=result_var, font=('Helvetica', 14)).grid(row=8, column=0, columnspan=2)

# Run the Tkinter event loop
root.mainloop()
